# Construct Reaction Network
In this notebook, I outline the development and concepts for developing a reaction network including primary species, secondary species, mineral kinetic reactions, and adsorption. Basically, this notebook covers all of the features of the model that you might need.

In [1]:
import pandas as pd
from pandas import DataFrame, Series
import numpy as np
from numpy.typing import NDArray
import potions as pt
import warnings 
warnings.filterwarnings("ignore")

## Define the reaction network
This reaction network will simulate a system with organic carbon and calcite.

### Primary species
In this network, the species involved are:
- Minerals: **Calcite** and **SOC** (soil organic carbon).
- Primary species: $\text{H}^+$, $\text{HCO3}^-$, and $\text{DOC}$
- Secondary Species: $\text{OH}^-$, $\text{CO2(aq)}$
- Adsorption Species:

In [2]:
cdbs = pt.ChemicalDatabase.load_default()

In [3]:
primary_species_names: list[str] = ["H+", "Ca++", "HCO3-", "DOC"]
secondary_species_names: list[str] = ["OH-", "CO2(aq)", "CO3--"]
mineral_names: list[str] = ["Calcite(s)", "SOC(s)"]
mineral_kinetic_names: list[str] = ["default", "test"]
adsorption_names: list[str] = ["XDOC"]

In [4]:
cdbs.monod_reactions["SOC"]

{'konza': MonodReaction(mineral_name='SOC', label='konza', rate_constant=-12.6, monod_terms={'SOC': 6e-10}, inhib_terms={})}

In [5]:
primary_species: list[pt.PrimaryAqueousSpecies] = cdbs.get_primary_aqueous_species(primary_species_names)
secondary_species: list[pt.SecondarySpecies] = cdbs.get_secondary_species(secondary_species_names)
mineral_species: list[pt.MineralSpecies] = cdbs.get_mineral_species(mineral_names)
mineral_reactions: pt.MineralKineticData = cdbs.get_mineral_reactions(mineral_names, mineral_kinetic_names)
adsorption_reactions: list[pt.ExchangeReaction] = cdbs.get_exchange_reactions(adsorption_names)

## Now, construct the matrices describing the chemical system

### Secondary species matrices
Calculating the secondary species requires the primary species, secondary species, and the adsorption reactions.

In [6]:
import importlib
importlib.reload(pt)

network = pt.ReactionNetwork(
    primary_aqueous=primary_species,
    mineral=mineral_species,
    secondary=secondary_species,
    mineral_kinetics=mineral_reactions,
    exchange=adsorption_reactions
)
species: DataFrame = network.species
species

,type
name,
H+,primary
Ca++,primary
HCO3-,primary
DOC,primary
X-,exchange
OH-,secondary
CO2(aq),secondary
CO3--,secondary
XDOC,exchange


#### Equilibrium stoichiometry matrix
The equlibrium stoichiometry matrix includes three groups of species with different roles
- Primary species: Mass and charge conservation
- Secondary species: Stoichiometry
- Exchange species: Mass and charge conservation

In this system, we have the following equilibrium equations:
$$
\begin{aligned}
H_{2}O &\leftrightarrow H^+ + OH^- \\
CO_2(aq) + H_2O &\leftrightarrow H^+ + HCO_3^- \\
XDOC^- &\leftrightarrow X^- + DOC
\end{aligned}
$$

In [7]:
sec_stoich_df: DataFrame = network.species
for sec in network.secondary:
    sec_stoich_df[f"{sec.name}"] = sec.stoichiometry
sec_stoich_df = sec_stoich_df.loc[sec_stoich_df["type"].isin(("primary", "secondary", "exchange"))].drop(columns=["type"]).fillna(0.0).T
print(f"Secondary species equilibrium matrix: \n{sec_stoich_df}")

sec_stoich_mat: NDArray = sec_stoich_df.to_numpy()

sec_eq_vec: NDArray = np.array([x.eq_consts[1] for x in network.secondary])
print(f"Log10 of Equilibrium constants: {sec_eq_vec}")

Secondary species equilibrium matrix: 
name      H+  Ca++  HCO3-  DOC   X-  OH-  CO2(aq)  CO3--  XDOC
OH-     -1.0   0.0    0.0  0.0  0.0 -1.0      0.0    0.0   0.0
CO2(aq)  1.0   0.0    1.0  0.0  0.0  0.0     -1.0    0.0   0.0
CO3--   -1.0   0.0    1.0  0.0  0.0  0.0      0.0   -1.0   0.0
Log10 of Equilibrium constants: [13.9951 -6.3447 10.329 ]


#### Mass and charge conservation
Now, we need to construct the mass and charge conservation matrices. These just come from the species matrix

In [8]:
network.exchange[0].stoichiometry

{'X-': 1.0, 'DOC': 1.0, 'XDOC': -1.0}

In [9]:
mass_stoich_df: DataFrame = network.species
for sec in network.secondary + network.exchange:
    mass_stoich_df[sec.name] = sec.stoichiometry

total_species: list[str] = mass_stoich_df.loc[(mass_stoich_df.type.isin(("primary", "mineral"))) | (mass_stoich_df.index == "X-")].index.tolist()
print(f"Total species: {total_species}")
mass_stoich_df = mass_stoich_df.loc[total_species].drop(columns="type").fillna(0.0)
primary_eye = DataFrame(np.eye(mass_stoich_df.shape[0]), columns=mass_stoich_df.index, index=mass_stoich_df.index)
mass_stoich_df = pd.concat([primary_eye, mass_stoich_df], axis=1)

rows = []
for i, row in mass_stoich_df.iterrows():
    if i == "H+":
        # Use charge balance for mass balance on 'H+'
        new_row = network.charges.loc[mass_stoich_df.columns]
        new_row.name = "Charge"
        rows.append(new_row)

    else:
        new_row = row.abs()
        new_row.name = f"Tot_{new_row.name}"
        rows.append(new_row)

mass_stoich_df = DataFrame(rows)[network.species_order]
mass_stoich_df

Total species: ['H+', 'Ca++', 'HCO3-', 'DOC', 'X-', 'Calcite(s)', 'SOC(s)']


,H+,Ca++,HCO3-,DOC,X-,OH-,CO2(aq),CO3--,XDOC,Calcite(s),SOC(s)
Charge,1.0,2.0,-1.0,0.0,-1.0,-1.0,0.0,-2.0,0.0,0.0,0.0
Tot_Ca++,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Tot_HCO3-,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0
Tot_DOC,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
Tot_X-,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
Tot_Calcite(s),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
Tot_SOC(s),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [10]:
eq = network.equilibrium_parameters

# Kinetic matrices

This is the other set of matrices used in this simulation and only involve the Monod and TST rate laws

In [11]:
tst = network.tst_params

In [13]:
tst.dep

name,H+,Ca++,HCO3-,DOC,X-,OH-,CO2(aq),CO3--,XDOC,Calcite(s),SOC(s)
OH-,-1.0,0.0,0.0,0.0,0.0,-1.0,0.0,0.0,0.0,0.0,0.0
CO2(aq),1.0,0.0,1.0,0.0,0.0,0.0,-1.0,0.0,0.0,0.0,0.0
CO3--,-1.0,0.0,1.0,0.0,0.0,0.0,0.0,-1.0,0.0,0.0,0.0
XDOC,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,-1.0,0.0,0.0
Calcite(s),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
SOC(s),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [13]:
network.mineral[0].stoichiometry

{'Ca++': 1.1, 'CO3--': 0.5, 'Calcite(s)': -1.0}

In [14]:
mineral_stoich_df: DataFrame = network.kinetic_species
for mineral in network.mineral:
    mineral_stoich_df[mineral.name] = mineral.stoichiometry

mineral_stoich_df = mineral_stoich_df.fillna(0.0)
mineral_stoich_df

,type,Calcite(s),SOC(s)
name,,,
H+,primary,0.0,0.00
Ca++,primary,1.1,0.00
HCO3-,primary,0.0,0.00
DOC,primary,0.0,0.55
OH-,secondary,0.0,0.00
CO2(aq),secondary,0.0,0.60
CO3--,secondary,0.5,0.00
Calcite(s),mineral,-1.0,0.00
SOC(s),mineral,0.0,-1.00


In [20]:
tst_dep_df: DataFrame = network.kinetic_species
for mineral in network.mineral:
    tst_dep_df[mineral.name] = 0.0

for (name, reaction) in network.mineral_kinetics.tst_reactions.items():
    tst_dep_df[name] = reaction.dependence
tst_dep_df = tst_dep_df.drop(columns=["type"]).fillna(0.0).T
tst_dep_df

name,H+,Ca++,HCO3-,DOC,OH-,CO2(aq),CO3--,Calcite(s),SOC(s)
Calcite(s),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
SOC(s),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [16]:
network.mineral_kinetics.monod_reactions["SOC(s)"]

MonodReaction(mineral_name='SOC(s)', label='test', rate_constant=-10.2, monod_terms={'SOC(s)': 6e-06}, inhib_terms={})

In [19]:
monod_df: DataFrame = network.kinetic_species
for mineral in network.mineral:
    monod_df[mineral.name] = 0.0
inhib_df = monod_df.copy()


for (name, reaction) in network.mineral_kinetics.monod_reactions.items():
    monod_df[reaction.mineral_name] = reaction.monod_terms
    inhib_df[reaction.mineral_name] = reaction.inhib_terms

monod_df: DataFrame = monod_df.drop(columns=["type"]).fillna(0.0).T
inhib_df: DataFrame = inhib_df.drop(columns=["type"]).fillna(0.0).T

print(f"Monod matrix: {monod_df}")
print(f"Inhibition matrix: {inhib_df}")

Monod matrix: name         H+  Ca++  HCO3-  DOC  OH-  CO2(aq)  CO3--  Calcite(s)    SOC(s)
Calcite(s)  0.0   0.0    0.0  0.0  0.0      0.0    0.0         0.0  0.000000
SOC(s)      0.0   0.0    0.0  0.0  0.0      0.0    0.0         0.0  0.000006
Inhibition matrix: name         H+  Ca++  HCO3-  DOC  OH-  CO2(aq)  CO3--  Calcite(s)  SOC(s)
Calcite(s)  0.0   0.0    0.0  0.0  0.0      0.0    0.0         0.0     0.0
SOC(s)      0.0   0.0    0.0  0.0  0.0      0.0    0.0         0.0     0.0
